In [4]:
#importing the libraries
import pandas as pd
import numpy as np
import os
from shapely.geometry import Point
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [5]:
RAW_PATH = "../data/NYPD_Arrests_Data_(Historic).csv"
PROCESSED_DIR = os.path.join("processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [6]:
df = pd.read_csv(RAW_PATH, low_memory=False)

print("Raw shape:", df.shape)
print(df.head())
print(df.info())
print(f"\nDataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

Raw shape: (5986025, 19)
   ARREST_KEY ARREST_DATE  PD_CD                         PD_DESC  KY_CD  \
0   298699285  12/31/2024  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
1   298663612  12/31/2024  922.0  TRAFFIC,UNCLASSIFIED MISDEMEAN  348.0   
2   298666333  12/31/2024  101.0                       ASSAULT 3  344.0   
3   298706306  12/31/2024  792.0      CRIMINAL POSSESSION WEAPON  118.0   
4   298700796  12/31/2024  203.0            TRESPASS 3, CRIMINAL  352.0   

                      OFNS_DESC    LAW_CODE LAW_CAT_CD ARREST_BORO  \
0                       ROBBERY  PL 1601001          F           Q   
1      VEHICLE AND TRAFFIC LAWS  VTL05110MU          M           B   
2  ASSAULT 3 & RELATED OFFENSES  PL 1200001          M           Q   
3             DANGEROUS WEAPONS  PL 265031B          F           Q   
4             CRIMINAL TRESPASS  PL 140100E          M           K   

   ARREST_PRECINCT  JURISDICTION_CODE AGE_GROUP PERP_SEX       PERP_RACE  \
0              113         

In [7]:
#check missing values
print("\n===== MISSING VALUES =====")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing)


===== MISSING VALUES =====
LAW_CAT_CD           24990
KY_CD                 9788
PD_DESC               9169
OFNS_DESC             9169
PD_CD                  884
LAW_CODE               196
AGE_GROUP               17
JURISDICTION_CODE       10
ARREST_BORO              8
Lon_Lat                  5
Longitude                5
Latitude                 5
X_COORD_CD               1
Y_COORD_CD               1
ARREST_KEY               0
PERP_RACE                0
PERP_SEX                 0
ARREST_DATE              0
ARREST_PRECINCT          0
dtype: int64


In [8]:
#fix missing values 
# Drop rows missing target LAW_CAT_CD
df = df.dropna(subset=["LAW_CAT_CD"])

# Fill offense description fields
df["PD_DESC"]  = df["PD_DESC"].fillna("UNKNOWN")
df["OFNS_DESC"] = df["OFNS_DESC"].fillna("UNKNOWN")
df["LAW_CODE"]  = df["LAW_CODE"].fillna("UNKNOWN")

# Fill numeric codes
df["PD_CD"] = df["PD_CD"].fillna(0)
df["KY_CD"] = df["KY_CD"].fillna(0)

# Fill demographic values
df["AGE_GROUP"]        = df["AGE_GROUP"].fillna("UNKNOWN")
df["ARREST_BORO"]      = df["ARREST_BORO"].fillna("UNKNOWN")
df["JURISDICTION_CODE"] = df["JURISDICTION_CODE"].fillna(-1)

# Coordinates + XY values
df["Latitude"]   = df["Latitude"].fillna(df["Latitude"].median())
df["Longitude"]  = df["Longitude"].fillna(df["Longitude"].median())
df["X_COORD_CD"] = df["X_COORD_CD"].fillna(df["X_COORD_CD"].median())
df["Y_COORD_CD"] = df["Y_COORD_CD"].fillna(df["Y_COORD_CD"].median())
df["Lon_Lat"]    = df["Lon_Lat"].fillna("")

# Print after cleaning
print("\n===== AFTER CLEANING =====")
print(df.isnull().sum().sort_values(ascending=False))


===== AFTER CLEANING =====
ARREST_KEY           0
JURISDICTION_CODE    0
Longitude            0
Latitude             0
Y_COORD_CD           0
X_COORD_CD           0
PERP_RACE            0
PERP_SEX             0
AGE_GROUP            0
ARREST_PRECINCT      0
ARREST_DATE          0
ARREST_BORO          0
LAW_CAT_CD           0
LAW_CODE             0
OFNS_DESC            0
KY_CD                0
PD_DESC              0
PD_CD                0
Lon_Lat              0
dtype: int64


In [9]:
#time feature
df["ARREST_DATE"] = pd.to_datetime(df["ARREST_DATE"], errors="coerce")

df['Year']        = df['ARREST_DATE'].dt.year
df['Month']       = df['ARREST_DATE'].dt.month
df['Day']         = df['ARREST_DATE'].dt.day
df['Hour']        = df['ARREST_DATE'].dt.hour
df['Weekday']     = df['ARREST_DATE'].dt.weekday
df['WeekOfYear']  = df['ARREST_DATE'].dt.isocalendar().week.astype(int)
df['Quarter']     = df['ARREST_DATE'].dt.quarter

df['Is_Weekend']     = df['Weekday'].isin([5, 6]).astype(int)
df['Pre_Post_2020']  = (df['Year'] >= 2020).astype(int)

df['During_COVID_Lockdown'] = df['ARREST_DATE'].apply(
    lambda d: 1 if (pd.Timestamp("2020-03-15") <= d <= pd.Timestamp("2020-06-30")) else 0
)

In [10]:
#age bucket
def age_bucket(age_group):
    if pd.isna(age_group):
        return "Unknown"

    s = str(age_group).strip()

    if s in ['15-17', '10-14', '<10']:
        return "15-17"
    if '18' in s or '19' in s or s == '18-24':
        return "18-24"
    if '25' in s or '30' in s or s == '25-44':
        return "25-44"
    if '45' in s or '50' in s or s == '45-64':
        return "45+"
    if '65' in s or s == '65+':
        return "45+"

    return "Other"

df['age_bucket'] = df['AGE_GROUP'].apply(age_bucket)

In [11]:
#sex and race
df['is_male']   = (df['PERP_SEX'].str.upper() == 'M').astype(int)
df['is_female'] = (df['PERP_SEX'].str.upper() == 'F').astype(int)

df['perp_race_clean'] = df['PERP_RACE'].fillna("UNKNOWN").str.upper().str.strip()

In [12]:
#offense group mapping
offense_map = {
    'ASSAULT': 'Assault',
    'HOMICIDE': 'Assault',
    'ROBBERY': 'Robbery',
    'BURGLARY': 'Burglary',
    'THEFT': 'Theft',
    'LARCENY': 'Theft',
    'AUTO': 'Vehicle',
    'MOTOR VEHICLE': 'Vehicle',
    'NARCOTIC': 'Drug',
    'DRUG': 'Drug',
    'WEAPON': 'Weapons',
    'FIREARM': 'Weapons',
    'WEAP': 'Weapons',
    'FRAUD': 'Fraud',
    'FORGERY': 'Fraud',
    'DISORDERLY CONDUCT': 'Disorderly Conduct',
    'PUBLIC PEACE': 'Disorderly Conduct',
    'PROSTITUTION': 'Public Safety',
    'DWI': 'Vehicle',
    'DWI/DUI': 'Vehicle',
    'SEX OFFENSE': 'Sexual Offense',
    'RAPE': 'Sexual Offense',
    'M V': 'Vehicle'
}

def map_offense(pd_desc, ofns_desc, law_code):
    sources = " ".join([
        str(pd_desc) if pd_desc == pd_desc else "",
        str(ofns_desc) if ofns_desc == ofns_desc else "",
        str(law_code) if law_code == law_code else ""
    ]).upper()

    for kw, group in offense_map.items():
        if kw in sources:
            return group

    # fallback
    if pd_desc and pd_desc == pd_desc:
        return pd_desc.strip().title()
    if ofns_desc and ofns_desc == ofns_desc:
        return ofns_desc.strip().title()

    return "Other"

df['offense_group'] = df.apply(
    lambda r: map_offense(
        r.get('PD_DESC', ""),
        r.get('OFNS_DESC', ""),
        r.get('LAW_CODE', "")
    ),
    axis=1
)

In [24]:
#violent anf felony flags
violent_keywords = ['ASSAULT', 'HOMICIDE', 'ROBBERY', 'RAPE', 'SEXUAL', 'WEAPON']

df['violent_flag'] = df['offense_group'].astype(str).str.upper().apply(
    lambda x: 1 if any(v in x for v in violent_keywords) else 0
)

df['felony_flag'] = df['LAW_CAT_CD'].astype(str).str.upper().str.startswith('F').astype(int)

In [14]:
#spatial feature scaling
coords = df[['Latitude', 'Longitude']].copy()
coords = coords.dropna()

if not coords.empty:
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(coords)
    idx = coords.index
    df.loc[idx, 'Latitude_scaled']  = scaled[:, 0]
    df.loc[idx, 'Longitude_scaled'] = scaled[:, 1]
else:
    df['Latitude_scaled']  = np.nan
    df['Longitude_scaled'] = np.nan

In [15]:
#precinct aggregation
if 'ARREST_PRECINCT' in df.columns:

    grp = df.groupby('ARREST_PRECINCT')
    precinct_summary = grp.agg(
        total_arrests=('ARREST_KEY', 'count'),
        felony_count=('felony_flag', 'sum'),
        violent_count=('violent_flag', 'sum'),
        avg_arrests_per_year=(
            'ARREST_KEY', 
            lambda s: s.count() / df['Year'].nunique() if df['Year'].nunique() else s.count()
        )
    ).reset_index()

    precinct_summary['felony_rate']  = precinct_summary['felony_count'] / precinct_summary['total_arrests']
    precinct_summary['violent_rate'] = precinct_summary['violent_count'] / precinct_summary['total_arrests']

    # offense mix
    top_offenses = df['offense_group'].value_counts().head(15).index.tolist()

    for o in top_offenses:
        counts = (
            df[df['offense_group'] == o]
            .groupby('ARREST_PRECINCT')['ARREST_KEY']
            .count()
            .rename(f"of_{o}_count")
        )
        precinct_summary = precinct_summary.merge(
            counts.reset_index(), on='ARREST_PRECINCT', how='left'
        )

    precinct_summary.fillna(0, inplace=True)

In [16]:
#borough summary
if 'ARREST_BORO' in df.columns:
    borough_grp = df.groupby('ARREST_BORO').agg(
        borough_total=('ARREST_KEY', 'count'),
        borough_felony=('felony_flag', 'sum')
    ).reset_index()

    borough_grp['borough_felony_share'] = borough_grp['borough_felony'] / borough_grp['borough_total']

In [17]:
#attach features back 
if 'precinct_summary' in locals():
    df = df.merge(
        precinct_summary[['ARREST_PRECINCT', 'total_arrests', 'felony_rate', 'violent_rate']],
        on='ARREST_PRECINCT', 
        how='left'
    )

In [18]:
#arrest density 
if 'total_arrests' in df.columns:
    total_all = df['ARREST_KEY'].nunique() if 'ARREST_KEY' in df.columns else len(df)
    df['precinct_arrest_density'] = df['total_arrests'] / total_all

In [19]:
#daily aggregates
df['arrest_date_day'] = df['ARREST_DATE'].dt.floor('D')

daily = (
    df.groupby('arrest_date_day')
      .agg(daily_arrests=('ARREST_KEY', 'count'))
      .reset_index()
      .sort_values('arrest_date_day')
)

daily['arrests_yesterday']   = daily['daily_arrests'].shift(1).fillna(0).astype(int)
daily['arrests_last_7_days'] = daily['daily_arrests'].rolling(7, min_periods=1).sum().shift(1).fillna(0)
daily['arrests_7day_avg']    = daily['daily_arrests'].rolling(7, min_periods=1).mean().shift(1).fillna(0)
daily['arrests_30day_avg']   = daily['daily_arrests'].rolling(30, min_periods=1).mean().shift(1).fillna(0)

df = df.merge(daily, on='arrest_date_day', how='left')

In [20]:
#basket for apriori
def build_basket_row(row):
    tags = []

    try:
        if pd.notna(row['offense_group']):
            tags.append("OFF:" + str(row['offense_group']).replace(" ", "_"))
        if pd.notna(row.get('ARREST_BORO')):
            tags.append("BORO:" + str(row['ARREST_BORO']).replace(" ", "_"))
        if pd.notna(row.get('ARREST_PRECINCT')):
            tags.append("PREC:" + str(int(row['ARREST_PRECINCT'])))
        if pd.notna(row.get('age_bucket')):
            tags.append("AGE:" + str(row['age_bucket']).replace(" ", "_"))
        if pd.notna(row.get('PERP_SEX')):
            tags.append("SEX:" + str(row['PERP_SEX']).upper())
        if pd.notna(row.get('PERP_RACE')):
            tags.append("RACE:" + str(row['perp_race_clean']).replace(" ", "_"))
        if pd.notna(row.get('Month')):
            tags.append("M:" + str(int(row['Month'])))
        if pd.notna(row.get('Weekday')):
            tags.append("WD:" + str(int(row['Weekday'])))

        if row.get('felony_flag', 0) == 1:
            tags.append("FELONY")
        if row.get('violent_flag', 0) == 1:
            tags.append("VIOLENT")

    except Exception:
        pass

    return list(set(tags))


df['basket'] = df.apply(build_basket_row, axis=1)
df['basket_str'] = df['basket'].apply(lambda items: ",".join(items))

In [25]:
#final engineered dataset
cols = [
    'ARREST_KEY','ARREST_DATE','Year','Month','Day','Hour','Weekday','WeekOfYear',
    'Is_Weekend','Pre_Post_2020','offense_group','violent_flag','felony_flag',
    'ARREST_BORO','ARREST_PRECINCT','Latitude','Longitude','Latitude_scaled','Longitude_scaled',
    'age_bucket','is_male','is_female','perp_race_clean',
    'daily_arrests','arrests_yesterday','arrests_last_7_days',
    'basket_str'
]

cols = [c for c in cols if c in df.columns]
engineered = df[cols]

print("Final dataset shape:", engineered.shape)
print("Feature engineering complete.")

Final dataset shape: (5961035, 27)
Feature engineering complete.


In [26]:
import os

# make sure the folder exists
os.makedirs("../processed", exist_ok=True)

# save the engineered dataframe
engineered.to_csv("../processed/NY_arrests_engineered_final.csv", index=False)

print("Saved: ../processed/NY_arrests_engineered_final.csv")


Saved: ../processed/NY_arrests_engineered_final.csv
